<a href="https://colab.research.google.com/github/csymapp/docs2Blog/blob/master/notebooks/docs2Blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Website Tools

Run this noteboook to update(push changes) to website once you have edited [the website config sheet](https://docs.google.com/spreadsheets/d/1-otVkXNn6i4tWGki9grXzc4DjG6TdQ_XWTBbJttlESU/edit#gid=512290428).

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

!pip install gdown

Download script and run
The script:
- Downloads dir template if missing (from github)
- Asks for ssh key (if missing) and writes to file
- Asks for docId and sheetId (if missing) and writes to file
- Processes the rest of the site

In [ ]:
%%bash

# rm -f docs2Blog.sh
# wget  --no-check-certificate --no-cache --no-cookies https://raw.githubusercontent.com/csymapp/docs2Blog/master/scripts/docs2Blog.sh -O docs2Blog.sh
# # curl -o docs2Blog.sh https://raw.githubusercontent.com/csymapp/docs2Blog/master/scripts/docs2Blog.sh
# {
# sudo ln -s "$(pwd)/docs2Blog.sh" /usr/local/bin/docs2Blog
# }
## No means found of downloading without return cached data using wget and curl. Best option is to just clone the repo
cd /tmp 
rm -rf docs2Blog
git clone https://github.com/csymapp/docs2Blog.git
cd docs2Blog/scripts 

chmod +x ./docs2Blog.sh
# cat ./scripts/docs2Blog.sh
ln -s "$(pwd)/docs2Blog.sh" /usr/local/bin/docs2Blog 2>/dev/null || true
# rm /usr/local/bin/docs2Blog && sudo ln -s "$(pwd)/scripts/docs2Blog.sh" /usr/local/bin/docs2Blog ## it link already exists
# rm /usr/local/bin/docs2Blog || sudo ln -s "$(pwd)/scripts/docs2Blog.sh" /usr/local/bin/docs2Blog ## if link does not exist
# sudo ln -s "$(pwd)/scripts/docs2Blog.sh" /usr/local/bin/docs2Blog

docs2Blog template
# docs2Blog ssh

Check if ssh key already exists. If not, ask for email address to use.

In [ ]:
import os
#from google.colab import files

file_path = '/content/drive/MyDrive/Publishing/Website-private/id_rsa'
if os.path.exists(file_path):
    print("The file exists.")
else:
    email = input("Please enter your email: ")
    os.environ['EMAIL'] = email
    !docs2Blog ssh "$EMAIL"
    ! mv "/content/drive/MyDrive/Publishing/Website-private/id_rsa.pub" "/content/drive/MyDrive/Publishing/Website-private/id_rsa.pub.txt"
    ! pandoc -o "/content/drive/MyDrive/Publishing/Website-private/id_rsa.pub.docx" "/content/drive/MyDrive/Publishing/Website-private/id_rsa.pub.txt"
    ! rm "/content/drive/MyDrive/Publishing/Website-private/id_rsa.pub.txt"
    print("Check your key in Publishing/Website-private/id_rsa.pub.docx, then")
    print("follow instructions in [Setting Up Key on GitHub](https://csymapp.github.io/docs2Blog/setting-up-key-github) to set it up")
    # files.download('/content/drive/MyDrive/Publishing/Website-private/id_rsa.pub')
    print("Once you have set up the key in github, come back here and:")
    input("Press and Key to Continue")

Now check if docId and sheetId are already set for the google spreadsheet with site configurations



In [ ]:
import os
#from google.colab import files
import yaml

def process_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            print(data)
            sheet_id = data.get('sheetId')
            doc_id = data.get('docId')
            if not doc_id:
                print("follow instructions in [Getting Doc Id](https://csymapp.github.io/docs2Blog/getting-doc-id) then enter it below.")
                doc_id = input("")
                os.environ['docId'] = doc_id
                ! echo "docId: $docId" >> /content/drive/MyDrive/Publishing/Website-private/siteConfig
            if not sheet_id:
                # instructions for how to get sheet id
                print("follow instructions in [Getting Sheet Id](https://csymapp.github.io/docs2Blog/getting-sheet-id) then enter it below.")
                sheet_id = input("")
                os.environ['sheetId'] = sheet_id
                ! echo "sheetId: $sheetId" >> /content/drive/MyDrive/Publishing/Website-private/siteConfig
            
            return sheet_id, doc_id
        except yaml.YAMLError as e:
            print("Error processing YAML file:", e)
            return None, None

file_path = '/content/drive/MyDrive/Publishing/Website-private/siteConfig'
if not os.path.exists(file_path):
    # create the file
    ! echo "abc: efg" > /content/drive/MyDrive/Publishing/Website-private/siteConfig

sheet_id, doc_id = process_yaml_file(file_path)
os.environ['docId'] = doc_id
os.environ['sheetId'] = str(sheet_id)

# print("sheetId:", sheet_id)
# print("docId:", doc_id)
!cd /tmp/docs2Blog/scripts && pwd && docs2Blog site
    